# 1. Objective
The report should consist of the following unique forms of adversarial gameplay:   
●  Discuss what you would do differently to enhance the system if there were no time 
or resource constraints. Include a proof of concept or demo to show that your idea 
is feasible.   
●  Share your personal  reflection, including your background, and explain how it 
influenced your strategies or provided an advantage in tackling the selected theme 
and coding tasks in this assignment.   
●  If you were to switch to the opposite  side of your current Google Colab project, 
outline the strategies you could use to attack (if you initially defended) or defend (if 
you  initially  attacked).  Provide a proof of concept or demo to demonstrate the 
concept.  

**Criteria for marks:**

| Criteria | Marks | Description |
| :--- | :--- | :--- |
| Strategic Depth & Feasibility | 6% | Describe a specific idea for a new feature, architecture, or mechanism. Provide a functional demo (e.g., code snippet, Colab notebook, video) to show the idea's feasibility. Explain the technical details and why it's a significant improvement. |
| Personal Reflection & Self-Awareness | 1.5% | Share relevant personal or professional experiences. Explain how these experiences influenced your strategies or provided a unique advantage. Reflect on what you learned about yourself and your skills during the project. |
| Adversarial/Defensive Creativity | 6% | Outline specific strategies for attacking (if you were a defender) or defending (if you were an attacker). Provide a functional demo (e.g., code snippet, Colab notebook, video) to showcase a key part of your new strategy. Justify why this strategy would be effective, based on your knowledge from the original project. |
| Analytical Rigor | 1.5% | Use data, logical arguments, and project findings to support every assertion. Provide a critical analysis of your work, other teams' approaches, and the system's vulnerabilities. Show a deep understanding by making non-obvious observations and drawing meaningful conclusions.

# 2. Explanation of work done so far
## Milestone 3 Concept Explanation: Aeroblade

> “For Milestone 3, we explored Aeroblade, which is a training-free detector for identifying AI-generated images.
>
> The key idea behind Aeroblade is very different from normal classifiers.
> Instead of training a neural network to tell real and fake apart, it uses the reconstruction ability of pretrained diffusion models, defines a distance metric between the input and reconstruced image in feature space, often using LPIPS which is Learned Perceptual Image Patch Similarity that compare images in a high level feature spaces that capture texture, color and structural pattern.
> Basically, how well an image can be recreated through the model’s latent space.
>
> The logic is simple but powerful:
>
> Real images usually have complex and irregular textures, so the diffusion models struggles to perfectly reconstruct them, giving a larger reconstruction error.
>
> Fake images, on the other hand, already exist on the same generative manifold that the diffusion models was trained on, so they are easier to reproduce, leading to smaller errors.
>
> By measuring this reconstruction difference — which we call the perceptual distance — Aeroblade can detect whether an image is likely real or synthetic without any additional training.”

---

### 1. VAE Round-Trip Preprocessing (“vaeround”)

* **What we did:**
    We introduced a VAE round-trip stage where each input image undergoes encoding and decoding using the Stable Diffusion VAE (AutoencoderKL) before being analyzed by Aeroblade. This process effectively reconstructs the image in the latent feature space, enforcing a form of “VAE regularization” at a resolution of 1024×1024.
* **Why it’s novel:**
    The original Aeroblade uses raw input images directly. By integrating a VAE round-trip step, we explore how well Aeroblade performs under latent-space reconstruction consistency — a rarely tested dimension of robustness.
* **Advantages:**
    * **Robustness testing:** Simulates real-world distortions, improving Aeroblade’s tolerance to compression and noise artifacts.
    * **Latent consistency check:** Highlights which image types (e.g., AI-generated vs. real) are more resilient or sensitive to latent reconstruction.
    * **Training-free enhancement:** Retains Aeroblade’s zero-training nature while expanding its analytical depth.
* **Professional justification:**
    This addition demonstrates scientific initiative — we didn’t alter Aeroblade’s model weights, but we extended its interpretive capability. It reflects a professional understanding of how to enhance model reliability through transform-level preprocessing, aligning with current best practices in AI forensics research.

### 2. End-to-End, One-Cell Automated Pipeline

* **What we did:**
    We built a unified end-to-end Jupyter cell that automates every step:
    repository cloning $\rightarrow$ environment setup $\rightarrow$ VAE preprocessing $\rightarrow$ Aeroblade execution $\rightarrow$ CSV post-processing $\rightarrow$ metric computation.
* **Why it’s novel:**
    The official Aeroblade repository provides modular scripts requiring multiple manual steps. We consolidated this into a single, reproducible pipeline that allows one-click execution and evaluation.
* **Advantages:**
    * **Efficiency:** Enables seamless batch processing and reproducibility for different image sets.
    * **Accessibility:** Allows future researchers or teammates to reproduce results without deep technical setup knowledge.
    * **Automation of analysis:** Integrates distance filtering, result aggregation, and visualization, reducing human error in post-processing.
* **Professional justification:**
    This demonstrates a professional engineering mindset — streamlining complex workflows into a clean, reproducible structure. In practical research or industry settings, automated pipelines are essential for scalability, reproducibility, and version control. It transforms a fragmented experimental process into a production-ready evaluation system.

### 3. Ground-Truth Auto-Extraction from File Paths

* **What we did:**
    We implemented an intelligent labeling system that automatically extracts the ground-truth class (real or fake) from file paths and filenames, rather than manually assigning labels.
* **Why it’s novel:**
    While simple conceptually, this solution eliminates tedious labeling work and ensures accurate alignment between dataset structure and result labeling — a feature not built into Aeroblade.
* **Advantages:**
    * **Automation:** Removes manual data handling, ensuring consistent labeling across experiments.
    * **Scalability:** Makes the system plug-and-play for new datasets with minimal human intervention.
    * **Error reduction:** Avoids mislabeled samples due to manual oversight, improving data integrity.
* **Professional justification:**
    This feature reflects attention to workflow design and human factors in research reproducibility. Automating ground-truth association is a hallmark of well-engineered ML pipelines, showing our focus on robust data management and efficient research methodology.

---

### 1. Data-Driven Decision Boundary ($\tau^*$) using Youden’s J Statistic

Instead of manually setting a threshold, we derived the optimal boundary analytically using Youden’s J index, which balances sensitivity and specificity.
$J = \text{True Positive Rate (TPR)} - \text{False Positive Rate (FPR)}$.

* **Why:** The original Aeroblade provides only reconstruction distances but no objective way to choose a cut-off for classification.
* **Advantage:** Our method transforms Aeroblade into a statistically optimized detector, ensuring fairer performance and reproducibility across different datasets.
* **Professional value:** Demonstrates a data-centric approach to model calibration, aligning with real-world deployment practices in AI forensics.

### 2. Policy-Based Thresholds ($\alpha = 0.10, 0.05, 0.01$)

We introduced configurable thresholds based on desired False Positive Rate (FPR) targets on real images.

* **Why:** In sensitive use cases (e.g., media forensics), tolerance for false alarms varies.
* **Advantage:** This approach provides a policy-driven control over detection strictness, making Aeroblade adaptable to different operational contexts.
* **Professional value:** Reflects foresight in practical system design, where trade-offs between accuracy and conservatism must be justified.

### 3. Comprehensive Metric Framework and Ablation Readiness

Our pipeline automatically calculates AUC, Precision, Recall, F1-score, and Accuracy, allowing detailed comparison across decision boundaries and datasets.

* **Why:** Quantitative metrics provide objective, transparent evaluation beyond single accuracy figures.
* **Advantage:** Enables nuanced understanding of Aeroblade’s strengths and weaknesses across real vs. fake domains.
* **Professional value:** Reflects scientific rigor and reproducibility, adhering to ML research reporting standards.

### 4. Explainability through Visualization

We created interpretable plots — unified distance histograms, ROC and PR curves, and metric-vs-threshold graphs — illustrating how decision logic changes with $\tau$.

* **Why:** Trust in AI forensics depends on clarity and interpretability.
* **Advantage:** These visuals help non-technical audiences understand why Aeroblade classifies an image as fake or real.
* **Professional value:** Showcases attention to model transparency and ethical usability.

In summary, our contribution goes beyond replication — it transforms Aeroblade into an analytically optimized, explainable, and policy-tunable detection framework. This reflects both creativity and applied research maturity.

---

## Result Presentations

Our results were presented in a manner emphasizing clarity, comprehensiveness, and professional reporting standards:

### 1. Structured and Readable Summary Tables

We presented key performance indicators in a well-formatted table with clear metric definitions and values.

* **Why:** Tabular presentation allows quick comprehension of critical findings.
* **Advantage:** Promotes clarity and professional readability, suitable for both technical and management-level stakeholders.
* **Professional value:** Mimics scientific presentation styles in research publications, emphasizing precision and interpretability.

### 2. Multi-Dimensional Visualization of Model Behavior

We employed ROC curves, Precision–Recall plots, and Threshold Sensitivity curves, each conveying different aspects of model performance.

* **Why:** A single plot cannot capture trade-offs between detection sensitivity, precision, and error rates.
* **Advantage:** Offers a holistic perspective of Aeroblade’s behavior under varying operating conditions.
* **Professional value:** Demonstrates mastery of data visualization and analytical communication, essential for AI evaluation work.

### 3. Unified Distribution Plot with Decision Zones

The histogram displaying the real vs. fake distance distributions with shaded decision zones provides an immediate visual interpretation of $\tau^*$.

* **Why:** Visual separation reinforces the statistical validity of the threshold.
* **Advantage:** Simplifies complex results into intuitive visual cues, improving transparency and stakeholder confidence.
* **Professional value:** Reflects expertise in communicating machine learning outcomes through evidence-based visuals.

### 4. Narrative Alignment Between Metrics and Visuals

The notebook integrates numeric results with plots and captions in a storytelling flow — from raw data, to threshold selection, to performance verification.

* **Why:** Ensures coherence and interpretability.
* **Advantage:** Audiences can follow the reasoning behind each conclusion without additional explanation.
* **Professional value:** Reflects attention to scientific communication standards — clear, logical, and reproducible.

In summary, our results presentation achieves a professional standard by combining quantitative rigor, visual clarity, and interpretive depth, ensuring that our findings are not only accurate but also accessible and defensible.

---

## Peer Targeting Impact

**Dataset & Boundary Selection:** We calibrated a single decision boundary ($\tau^*$) on disjoint real and fake datasets by maximizing Youden’s J ($\text{TPR} - \text{FPR}$) over the absolute Aeroblade reconstruction distances. This yields a training-free, statistically optimised cutoff that balances missed fakes and false alarms.

**Peer Targeting Evaluation:** We then froze $\tau^*$ and evaluated on an unseen peer-generated fake set. All peer images were classified as fake at $\tau^*$, i.e., $\text{TPR}_{\text{peer}} = 1.00$, demonstrating strong generalisation of the boundary to adversarial sources. ROC/PR curves and metric-vs-threshold plots (with $\tau^*$ marked) confirm the operating point is robust rather than cherry-picked. Distribution plots further show peer fakes cluster within the “fake” zone, despite producing relatively small reconstruction errors, indicating they lie close to but still detectably off the VAE’s natural image manifold.

We observed that peer-generated images exhibited consistently small reconstruction errors, indicating that they lie closer to the latent manifold of the Stable Diffusion VAE.
This pattern suggests that the fakes are more easily recognized by Aeroblade, since smaller |distance| values correspond to images that are structurally aligned with the generative model’s internal distribution.
Consequently, our optimized Youden-based decision boundary ($\tau^*$) effectively separated these images as fake, confirming both the validity and robustness of our calibration strategy.

# 3 Future work/Current shortcomings of project/Other notes (in short points, in no particular order)

1. Enhancements if there were no constraints:  
- Aeroblade does its deconstruction-reconstruction on 3 different types of diffusion models, stable diffusion 1, 2 and kadinsky. I don't think this is an efficient use of processing power - these 3 give similar results, and the only real help of aggregating these 3 is that if an input image is made by one of these diffusion models, it becomes very obviously distinguished. But, we want a generalizable detector that works on every attack, hence my adding the VAE pre-processing step, so using 3 diffusion models is redundant. We should pick just one, and instead aggregate it with some other discriminating features like perhaps spectral frequencies, entropy, etc, anything that isn't auto-encoder reconstruction error, that way we can combine multiple features to identify fake images rather than just one feature. I don't have a proof of concept for this, and will not run a demo, but this should be easily mathematically and logically sound.  

2. Reflection and background:
As a masters of data science student, and previously a robotics engineer from electrical and computer systems engineering, it leads me to always consider: what's actually going in to our model? What kind of features would go in, what features does the model actually use, and how does it use these features to make its predictions? Working on this project also made me realize how much of a pain dealing with dependency issues are and how much documentation helps other people picking up your project.

3. Attacker strategies:  
As aeroblade classifies a real image as having a higher reconstruction error after deconstructing and reconstructing through a diffusion model, and as of our modifications, being after first reconstructed with a VAE, we need some form of surprisal that survives a VAE reconstruction. The most obvious choice for entropy, is, of course, adding gaussian noise, but this also comes with the caveat that adding noise makes your image look poorer in quality, something usually undesirable. This leads me to imagine that a FGSM attack might possibly work quite well here.
